In [1]:
!pip install ipywidgets requests pandas matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00


In [73]:
# Enable ipywidgets in Colab
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output, Image, HTML
import pandas as pd
import matplotlib.pyplot as plt
import getpass
from PIL import Image as PILImage
from io import BytesIO



In [8]:
import getpass

# Prompt the user to enter their OpenWeatherMap API key securely
API_KEY = getpass.getpass('Enter your OpenWeatherMap API Key: ')


Enter your OpenWeatherMap API Key: ··········


In [86]:

def get_weather_data(city_name, units='metric'):
    """
    Fetches current weather data for the specified city.
    """
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city_name,
        'appid': API_KEY,
        'units': units
    }
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        if response.status_code == 200:
            return data
        else:
            return data  # Return the error message
    except Exception as e:
        return {'error': str(e)}

def get_forecast_data(city_name, units='metric'):
    """
    Fetches 5-day weather forecast data for the specified city.
    """
    base_url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {
        'q': city_name,
        'appid': API_KEY,
        'units': units
    }
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        if response.status_code == 200:
            return data
        else:
            return data  # Return error message
    except Exception as e:
        return {'error': str(e)}


def display_weather(b):
    """
    Fetches and displays weather information based on user input.
    """
    with weather_output:
        clear_all_outputs()
        city = city_input.value.strip()
        units = unit_dropdown.value
        if not city:
            print("🚫 Please enter a city name.")
            return
        data = get_weather_data(city, units)
        if 'error' in data:
            print(f"⚠️ An error occurred: {data['error']}")
            return
        if data.get('cod') != 200:
            print(f"❌ Error: {data.get('message', 'City not found.')}")
            return
        # Extract relevant data

        city_name = data.get('name')
        country = data['sys'].get('country')
        weather_desc = data['weather'][0].get('description').title()
        temperature = data['main'].get('temp')
        humidity = data['main'].get('humidity')
        wind_speed = data['wind'].get('speed')
        icon_code = data['weather'][0].get('icon')
        icon_url = f"http://openweathermap.org/img/wn/{icon_code}@2x.png"
        # Determine temperature unit symbol
        temp_unit = '°C' if units == 'metric' else '°F'
        # Display the data

        html_content = f"""
            <div style="font-family: Arial; color: #4A90E2;">
                <img src="{icon_url}" alt="Weather Icon" width="100">
                <h2>📍 Location: {city_name}, {country}</h2>
                <p><strong>🌡️ Temperature:</strong> {temperature}{temp_unit}</p>
                <p><strong>☁️ Weather:</strong> {weather_desc}</p>
                <p><strong>💧 Humidity:</strong> {humidity}%</p>
                <p><strong>💨 Wind Speed:</strong> {wind_speed} m/s</p>
            </div>
        """
        display(HTML(html_content))

def path_to_image_html(path):
    """
    Converts an image URL to an HTML <img> tag.

    Parameters:
    - path (str): URL of the image.

    Returns:
    - str: HTML <img> tag with the image.
    """
    return f'<img src="{path}" width="50" height="50">'

def display_forecast(b):
    """
    Fetches and displays a 5-day weather forecast based on user input.
    Clears previous outputs before displaying new data.
    """
    clear_all_outputs()  # Clear all previous outputs
    with forecast_output:

        city = city_input.value.strip()
        units = unit_dropdown.value
        if not city:
            print("🚫 Please enter a city name.")
            return
        data = get_forecast_data(city, units)
        if 'error' in data:
            print(f"⚠️ An error occurred: {data['error']}")
            return
        if data.get('cod') != "200":
            print(f"❌ Error: {data.get('message', 'City not found.')}")
            return
        # Extract relevant data
        forecast_list = data.get('list', [])
        if not forecast_list:
            print("❌ No forecast data available.")
            return
        # Process data to get daily forecasts
        forecast_dict = {}
        for entry in forecast_list:
            date_txt = entry['dt_txt']
            date = date_txt.split(' ')[0]
            time = date_txt.split(' ')[1]
            if date not in forecast_dict:
                forecast_dict[date] = {}
            forecast_dict[date][time] = entry
        # Create a DataFrame for the forecast
        forecast_data = []
        for date, times in forecast_dict.items():
            # Get temperatures at 12:00:00
            noon_entry = times.get('12:00:00')
            if noon_entry:
                temp = noon_entry['main']['temp']
                weather_desc = noon_entry['weather'][0]['description'].title()
                icon_code = noon_entry['weather'][0]['icon']
                forecast_data.append({
                    'Date': date,
                    'Temperature': temp,
                    'Weather': weather_desc,
                    'Icon': f"https://openweathermap.org/img/wn/{icon_code}@2x.png"
                })
        # Convert to DataFrame
        forecast_df = pd.DataFrame(forecast_data)
        if forecast_df.empty:
            print("❌ No midday forecast data available.")
            return
        # Display the forecast table
        print("### 5-Day Forecast")
        styled_forecast_df = forecast_df[['Date', 'Temperature', 'Weather', 'Icon']].style.set_properties(**{
    'text-align': 'center',
    'font-family': 'Arial'
}).format({'Icon': path_to_image_html})
        display(styled_forecast_df)
        # # Display weather icons
        # print("\n### Weather Icons")
        # icons = [f'<img src="{icon}" alt="Weather Icon" style="width:50px;height:50px;">' for icon in forecast_df['Icon']]
        # icon_html = '<br>'.join(icons)
        # display(HTML(icon_html))


def display_temperature_trend(b):
    """
    Displays a simulated temperature trend chart.
    """
    with trend_output:
        clear_all_outputs()
        city = city_input.value.strip()
        units = unit_dropdown.value
        if not city:
            print("🚫 Please enter a city name.")
            return
        data = get_weather_data(city, units)
        if 'error' in data:
            print(f"⚠️ An error occurred: {data['error']}")
            return
        if data.get('cod') != 200:
            print(f"❌ Error: {data.get('message', 'City not found.')}")
            return
        temperature = data['main'].get('temp')
        # Simulate temperature trend data
        # For demonstration, we'll generate random data around the current temperature
        import random
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        temps = [temperature + random.uniform(-3, 3) for _ in days]
        # Determine temperature unit symbol
        temp_unit = '°C' if units == 'metric' else '°F'
        # Plot the temperature trend
        plt.figure(figsize=(10,5))
        plt.plot(days, temps, marker='o', linestyle='-', color='#FF7F50')
        plt.title(f"Temperature Trend for {city}")
        plt.xlabel("Day of the Week")
        plt.ylabel(f"Temperature ({temp_unit})")
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

def clear_all_outputs():
    """
    Clears all output widgets to ensure a clean display.
    """
    with weather_output:
        clear_output()
    with trend_output:
        clear_output()
    with forecast_output:
        clear_output()

In [87]:
# Create a Text widget for city input
title_html = widgets.HTML(value="""
    <div style="text-align: center; font-family: Arial; color: #4A90E2;">
        <h1>🌤️ Weather App</h1>
    </div>
""")

# Create a Text widget for city input
city_input = widgets.Text(
    value='',
    placeholder='Enter city name, e.g., London, New York',
    description='City:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

# Create a Dropdown widget for unit selection
unit_dropdown = widgets.Dropdown(
    options=[('Celsius (°C)', 'metric'), ('Fahrenheit (°F)', 'imperial')],
    value='metric',
    description='Units:',
    disabled=False,
    layout=widgets.Layout(width='30%')
)

# Create a Button widget to trigger the search
search_button = widgets.Button(
    description='Get Weather',
    disabled=False,
    button_style='success',  # Options: 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to get current weather',
    icon='cloud'  # (FontAwesome names without the `fa-` prefix)
)

forecast_button = widgets.Button(
    description='Get Forecast',
    disabled=False,
    button_style='info',
    tooltip='Click to get 5-day weather forecast',
    icon='calendar'  # FontAwesome icon
)

trend_button = widgets.Button(
    description='Show Temp Trend',
    disabled=False,
    button_style='info',
    tooltip='Click to show temperature trend',
    icon='chart-line'
)
# Create an Output widget to display weather information
weather_output = widgets.Output()

# Create an Output widget to display temperature trend
trend_output = widgets.Output()

forecast_output = widgets.Output()

# Create a Button widget to show temperature trend




In [88]:
# Link the search button to the display_weather function
search_button.on_click(display_weather)

# Link the trend button to the display_temperature_trend function
trend_button.on_click(display_temperature_trend)

forecast_button.on_click(display_forecast)

# Link the export button to the export_weather_data function

# Arrange the search section (City input and Unit selection)
search_section = widgets.HBox([
    city_input,
    unit_dropdown
])

# Arrange the buttons horizontally
# Arrange the buttons horizontally with centered alignment
button_section = widgets.HBox([
    search_button,
    forecast_button,
    trend_button
], layout=widgets.Layout(
    justify_content='center',  # Centers the buttons horizontally
    align_items='center',      # Centers the buttons vertically within the HBox
    width='100%'               # Makes the HBox span the full width for better centering
))

# Arrange the entire app layout using VBox and HBox
app_ui = widgets.VBox([
    title_html,
    search_section,
    button_section,
    weather_output,
    forecast_output,
    trend_output
])

# Display the app
display(app_ui)
